In [1]:
#libraries 
import numpy as np
import pandas as pd
import re
import os


#preprocessing
import string
from time import time  
from nltk.corpus import stopwords #remove stop words
chachedWords = stopwords.words('english')

#cosine similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import math
from collections import Counter


Base: https://medium.com/@adriensieg/text-similarities-da019229c894

REF: 
    
https://arxiv.org/pdf/1301.3781.pdf
    
https://www.aclweb.org/anthology/N19-1181.pdf
    
http://proceedings.mlr.press/v37/kusnerb15.pdf

https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/37842.pdf

https://arxiv.org/pdf/1609.08144.pdf

In [2]:
#df = pd.read_csv(r"C:\Users\hirom\OneDrive - NOVAIMS\NOVA IMS\OneDrive_1_25-02-2021\Text Mining\Project\corpus\de-en\scores.csv")
df = pd.read_csv(r"C:\Users\hirom\Documents\GitHub\textmining_project\test\zh-en\scores.csv")
df

,source,reference,translation
0,已经批准筹建的，暂停批准开业,"Where the preparation has been approved, the a...",Approval of opening on these establishments wi...
1,王丰源在首发式发言中说，来美国前想找本书看看别人的经验，但他翻遍新华书店没找到关于留学美国中...,"In his opening speech, Mr. Wang said he wanted...",Wang Fengyuan spoke at the launch of his new b...
2,“如果你不致力于创造透明文化，你会失去人才，”维特拉诺说道。,"""if you're not committed to creating a culture...","""If you're not committed to creating a culture..."
3,不过前提是多国联军先停止对也门的袭击。,"The premise, however, is that the coalition fo...","However, the premise is that the multinational..."
4,“在此之前，我和前男友住在骑士桥的一个更大的房子里，”乔安妮说道。,"""before that, my ex and I lived in a bigger ho...","""Before this, I was living with my ex in Knigh..."
...,...,...,...
25347,被判定年龄达到 18 岁以上的案例占据较高比例并不一定表示这些避难者就谎报了年龄。,The high proportion of cases where the age of ...,The high proportion of cases judged to be over...
25348,“我们的问题将在不久的将来通过法院解决”，科林写道。,“Our problems will be resolved by the courts i...,"""Our matter will be resolved by the courts in ..."
25349,《Riodoce》的全体员工都很怀念瓦尔迪兹，怀念他像一个码头工人一样咒骂不休，怀念这位从不...,"All Riodock employees miss Valdez, remembering...","The Riodoce staff misses Valdez, the jokester ..."
25350,观众戴上面具，成为故事中幽灵一般的存在，这是《不眠之夜》最大的游戏规则。,The spectators wear the mask and become the ge...,"Audiences put on masks, and exist like a spiri..."


## Preprocessing 

In [3]:
def preprocessing2(sentence):
    return " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in sentence.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in chachedWords)
     

In [4]:
df['p_reference'] = df.apply(lambda x:preprocessing2(x['reference']),axis=1)

In [5]:
df['p_translation'] = df.apply(lambda x:preprocessing2(x['translation']),axis=1)

In [6]:
def delete_sw(sentence):
    stop_words = stopwords.words('english')
    processed = sentence.split()
    clean = [w for w in processed if w not in chachedWords]
    return clean 

In [7]:
df_cp = df.copy()

In [8]:
df_cp['w_reference'] = df_cp.apply(lambda x:delete_sw(x['p_reference']),axis=1)

In [9]:
df_cp['w_translation'] = df_cp.apply(lambda x:delete_sw(x['p_translation']),axis=1)

## Embedding

https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/

Doc2Vec

In [10]:
# import
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(df_cp['w_translation'])]
tagged_data

C:\Users\hirom\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[TaggedDocument(words=['approval', 'opening', 'establishments', 'suspended'], tags=[0]),
 TaggedDocument(words=['wang', 'fengyuan', 'spoke', 'launch', 'new', 'book', 'saying', 'coming', 'united', 'states', 'wanted', 'find', 'book', 'get', 'know', 'experience', 'others', 'searched', 'xinhua', 'bookstore', 'find', 'book', 'studying', 'american', 'middle', 'school'], tags=[1]),
 TaggedDocument(words=['committed', 'creating', 'culture', 'transparency', 'going', 'lose', 'people', 'vitrano', 'said'], tags=[2]),
 TaggedDocument(words=['however', 'premise', 'multinational', 'coalition', 'stop', 'attack', 'yemen', 'first'], tags=[3]),
 TaggedDocument(words=['living', 'ex', 'knightsbridge', 'much', 'bigger', 'space', 'joanne', 'says'], tags=[4]),
 TaggedDocument(words=['even', 'wild', 'tropical', 'conditions', 'appropriate', 'flowers', 'blooming', 'money', 'trees', 'rare', 'said', 'chance', 'flowers', 'blooming', 'one', 'thousand'], tags=[5]),
 TaggedDocument(words=['end', 'year', 'leaders', 'ne

In [ ]:
## Train doc2vec model
model_Doc2Vec = Doc2Vec(tagged_data, vector_size = 1024, window = 3, min_count = 1.5, epochs = 100)



In [ ]:
def Doc2Vec(sentece):
    x = model_Doc2Vec.infer_vector(sentece)    
    return x
    

In [ ]:
#embedding = df_cp.apply(lambda x:Doc2Vec(x['w_reference']), axis=1)

In [ ]:
#pd.DataFrame(embedding).to_csv('embedding.csv')

## Similarity 

## Doc2Vec + Cosine Similarity 

In [ ]:
from scipy import spatial
def cosinesimilarity(ref,trans):
    t = model_Doc2Vec.infer_vector(trans)
    r = model_Doc2Vec.infer_vector(ref)
    result = 1 - spatial.distance.cosine(t,r)
    return result

REF: https://www.machinelearningplus.com/nlp/cosine-similarity/

The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance (due to the size of the document), chances are they may still be oriented closer together. The smaller the angle, higher the cosine similarity.

**Interpretation** : A cosine value of 0 means that the two vectors are at 90 degrees to each other (orthogonal) and have no match. The closer the cosine value to 1, the smaller the angle and the greater the match between vectors. 

In [ ]:
df_cp['metric'] = df_cp.apply(lambda x:cosinesimilarity(x['w_reference'],x['w_translation']), axis=1)

In [ ]:
df_cp

In [ ]:
df_final = df_cp.drop(columns=['p_reference', 'p_translation','w_reference','w_translation'])

In [ ]:
df_final